# 🧪 Hands-On Lab for Section 2: Foundations of RAG Architecture

**Lab Title:**
Implementing the Retrieval and Generation Pipeline with FAISS and OpenAI LLM

## Objective:
By the end of this lab, learners will:
- Build a retrieval subsystem that finds relevant text from a vector database.
- Build a generation subsystem that uses LLMs to create context-aware answers.
- Combine both to form a working RAG mini-architecture.

## Install dependencies

In [83]:
!pip install openai faiss-cpu tiktoken pandas

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## Import libraries

In [84]:
import numpy as np
import faiss
import os
from dotenv import load_dotenv
from openai import OpenAI

# Load environment variables from .env file
load_dotenv()

# Get API key from environment variable
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")

if not OPENROUTER_API_KEY:
    raise ValueError("OPENROUTER_API_KEY not found! Please set it in your .env file")

# Initialize OpenRouter client
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY
)

print("✅ OpenRouter client initialized")

✅ OpenRouter client initialized


## 🧩 Part 1 – The Retrieval Process

**Goal:** Understand how embeddings are created, stored, and searched using semantic similarity.

### Create a mini-dataset

In [85]:
knowledge_base = [
    "RAG stands for Retrieval-Augmented Generation.",
    "In RAG, relevant context is fetched before generating answers.",
    "Vector databases such as FAISS store embeddings for fast search.",
    "Embeddings represent text meaning as numerical vectors.",
    "Combining retrieval and generation improves accuracy and factuality."
]

print("Knowledge base created with", len(knowledge_base), "documents")

Knowledge base created with 5 documents


### Generate embeddings and store in FAISS

In [86]:
# Install sentence-transformers for local embeddings
%pip install sentence-transformers

from sentence_transformers import SentenceTransformer

# Initialize a local embedding model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

def get_embedding(text):
    """Generate embeddings using a local sentence transformer model"""
    return embedding_model.encode(text).tolist()

# Generate embeddings for all documents in knowledge base
print("Generating embeddings...")
vectors = [get_embedding(text) for text in knowledge_base]

# Create FAISS index
dimension = len(vectors[0])
index = faiss.IndexFlatL2(dimension)
index.add(np.array(vectors).astype("float32"))

print(f"✅ FAISS index created with {index.ntotal} vectors of dimension {dimension}")


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Generating embeddings...
✅ FAISS index created with 5 vectors of dimension 384


### Test retrieval

In [87]:
query = "How does RAG improve AI accuracy?"

# Get embedding for the query
query_vector = get_embedding(query)

# Search for top 2 most similar documents
D, I = index.search(np.array([query_vector]).astype("float32"), k=2)

# Retrieve the matching documents
retrieved_docs = [knowledge_base[i] for i in I[0]]

print("Top matches:\n")
for idx, doc in enumerate(retrieved_docs, 1):
    print(f"{idx}. {doc}")

Top matches:

1. RAG stands for Retrieval-Augmented Generation.
2. In RAG, relevant context is fetched before generating answers.


## 🧠 Part 2 – The Generation Process

**Goal:** Use an LLM to generate context-aware answers based on retrieved data.

### Combine retrieved context and query

In [88]:
context = " ".join(retrieved_docs)
prompt = f"Use the context below to answer precisely.\n\nContext:\n{context}\n\nQuestion: {query}"

print("Prompt sent to LLM:")
print("-" * 60)
print(prompt)
print("-" * 60)

Prompt sent to LLM:
------------------------------------------------------------
Use the context below to answer precisely.

Context:
RAG stands for Retrieval-Augmented Generation. In RAG, relevant context is fetched before generating answers.

Question: How does RAG improve AI accuracy?
------------------------------------------------------------


### Generate the answer

In [89]:
try:
    response = client.chat.completions.create(
        model="qwen/qwen-2-7b-instruct:free",
        messages=[{"role": "user", "content": prompt}]
    )
    
    print("Generated Answer:\n")
    print(response.choices[0].message.content)
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("\nPlease verify:")
    print("1. Your OPENROUTER_API_KEY in the .env file")
    print("2. Get a fresh key from https://openrouter.ai/keys")
    print("3. Restart the kernel and re-run from the beginning")

❌ Error: Error code: 404 - {'error': {'message': 'No endpoints found for qwen/qwen-2-7b-instruct:free.', 'code': 404}, 'user_id': 'user_2wfJyNuF5yzdHhNuld6ltJBaBZP'}

Please verify:
1. Your OPENROUTER_API_KEY in the .env file
2. Get a fresh key from https://openrouter.ai/keys
3. Restart the kernel and re-run from the beginning


## ⚙️ Part 3 – Putting It Together

**Goal:** Build a complete RAG function that takes a user question and returns a contextual answer.

### Define a complete RAG pipeline function

In [90]:
def rag_pipeline(question, k=2):
    """
    Complete RAG pipeline: Retrieval + Generation
    
    Args:
        question: User's question
        k: Number of top documents to retrieve
    
    Returns:
        Generated answer based on retrieved context
    """
    # Step 1: Retrieve relevant documents
    q_vec = get_embedding(question)
    D, I = index.search(np.array([q_vec]).astype("float32"), k=k)
    retrieved_docs = [knowledge_base[i] for i in I[0]]
    context = " ".join(retrieved_docs)
    
    print(f"📚 Retrieved {k} relevant documents:")
    for idx, doc in enumerate(retrieved_docs, 1):
        print(f"  {idx}. {doc}")
    print()
    
    # Step 2: Create prompt with context
    prompt = f"Context:\n{context}\n\nQuestion: {question}"
    
    # Step 3: Generate answer using OpenRouter
    response = client.chat.completions.create(
        model="qwen/qwen-2-7b-instruct:free",
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

print("✅ RAG pipeline function ready!")

✅ RAG pipeline function ready!


### Ask your AI

In [91]:
print("Question 1: What are vector databases used for?")
print("-" * 60)
# First, let's update the rag_pipeline function to use a working model
def rag_pipeline_fixed(question, k=2):
	"""
	Complete RAG pipeline: Retrieval + Generation (with working model)
	
	Args:
		question: User's question
		k: Number of top documents to retrieve
	
	Returns:
		Generated answer based on retrieved context
	"""
	# Step 1: Retrieve relevant documents
	q_vec = get_embedding(question)
	D, I = index.search(np.array([q_vec]).astype("float32"), k=k)
	retrieved_docs = [knowledge_base[i] for i in I[0]]
	context = " ".join(retrieved_docs)
	
	print(f"📚 Retrieved {k} relevant documents:")
	for idx, doc in enumerate(retrieved_docs, 1):
		print(f"  {idx}. {doc}")
	print()
	
	# Step 2: Create prompt with context
	prompt = f"Context:\n{context}\n\nQuestion: {question}"
	
	# Step 3: Generate answer using a working model
	try:
		response = client.chat.completions.create(
			model="meta-llama/llama-3.2-3b-instruct:free",  # Using a different free model
			messages=[{"role": "user", "content": prompt}]
		)
		return response.choices[0].message.content
	except Exception as e:
		print(f"Error with meta-llama model: {e}")
		try:
			# Fallback to another model
			response = client.chat.completions.create(
				model="microsoft/phi-3-mini-128k-instruct:free",
				messages=[{"role": "user", "content": prompt}]
			)
			return response.choices[0].message.content
		except Exception as e2:
			return f"Error generating response: {e2}"

answer1 = rag_pipeline_fixed("What are vector databases used for?")
print(answer1)
print("\n" + "=" * 60 + "\n")

print("Question 2: Explain the main advantage of RAG.")
print("-" * 60)
answer2 = rag_pipeline_fixed("Explain the main advantage of RAG.")
print(answer2)

Question 1: What are vector databases used for?
------------------------------------------------------------
📚 Retrieved 2 relevant documents:
  1. Vector databases such as FAISS store embeddings for fast search.
  2. Embeddings represent text meaning as numerical vectors.

Vector databases, such as FAISS (Facebook AI Similarity Search), are used for efficient similarity search and retrieval of dense vector embeddings. These embeddings represent text, images, or other high-dimensional data as numerical vectors, allowing for fast and accurate querying of similar data points.

Some common use cases for vector databases include:

1. **Text Search and Recommendation**: Vector databases are used in search engines, recommender systems, and content recommendation platforms to find similar documents, articles, or products based on their vector embeddings.
2. **Image Retrieval**: Vector databases are used in image retrieval applications, such as object detection, image classification, and facia

## 🧭 Bonus Challenge

Try these experiments:

1. **Replace the toy dataset** with your own text (PDF excerpts, company docs, or Wikipedia articles).
2. **Experiment with different k values** (top-k retrieval) to see how it affects the quality of answers.
3. **Compare responses with and without retrieval** to see the impact on factual accuracy.

### Example: Compare with and without RAG

In [92]:
test_question = "What does RAG stand for?"

# Without RAG (direct query)
print("WITHOUT RAG (No Context):")
print("-" * 60)
response_no_rag = client.chat.completions.create(
    model="qwen/qwen-2-7b-instruct:free",
    messages=[{"role": "user", "content": test_question}]
)
print(response_no_rag.choices[0].message.content)

print("\n" + "=" * 60 + "\n")

# With RAG (context-aware)
print("WITH RAG (Context-Aware):")
print("-" * 60)
response_with_rag = rag_pipeline(test_question)
print(f"\n🎯 Answer: {response_with_rag}")

print("\n" + "=" * 60)
print("\n✅ Notice how RAG provides more specific, context-grounded answers!")

WITHOUT RAG (No Context):
------------------------------------------------------------


NotFoundError: Error code: 404 - {'error': {'message': 'No endpoints found for qwen/qwen-2-7b-instruct:free.', 'code': 404}, 'user_id': 'user_2wfJyNuF5yzdHhNuld6ltJBaBZP'}

## 🎯 Summary

You've successfully built a complete RAG system! Here's what you accomplished:

1. ✅ **Retrieval**: Created embeddings and stored them in FAISS for semantic search
2. ✅ **Generation**: Used an LLM to generate context-aware answers
3. ✅ **Integration**: Combined both into a single `rag_pipeline()` function
4. ✅ **Comparison**: Saw the difference between RAG and non-RAG responses

This is your first fully functional RAG mini-system!